# Network Analysis

StrainDesign provides canonical functions for maximizing and minimizing metabolic fluxes in network. The output format is identical to the ones of COBRApy's functions.

## Flux optimization (FBA/pFBA)

Flux balance analysis is a single linear program that optimizes a flux rate under the given steady-state network constraints. For the case of a growth-rate maximization, the problem is written as:
$$\text{maximize}\:\:\: v_{growth} \\ \text{subject to} \\ \mathbf{S\, v = 0} \\ \mathbf{lb \le v \le ub}$$
Where $ \mathbf{S}$ is the stoichiometric matrix, $\mathbf{v}$ is the vector of metabolic flux rates and $\mathbf{lb}$ and $\mathbf{ub}$ are the physiological lower and upper bounds of each flux rate. $\mathbf{S\, v = 0}$ represents all steady-state constraints, and $\mathbf{lb \le v \le ub}$ the flux ranges.

All of the following computation examples will require the COBRApy and the StrainDesign package. Here, we load both packages and the e_coli_core model from Bigg:

In [9]:
import cobra
import straindesign as sd
model = cobra.io.load_model('e_coli_core')

An FBA is launched by a single function call. By default the model's objective function is optimized. The function returns a solution object, in which the objective value and the fluxes are stored in `solution.objective_value` and `solution.fluxes`.

In [8]:
solution = sd.fba(model)

print(f"Maximum growth: {solution.objective_value}.")

Maximum growth: 0.8739215069684305.


You may also use a custom objective and change the optimization sense:

In [11]:
solution = sd.fba(model,obj='GLCpts',obj_sense='minimize')

print(f"Minimum flux through GLCpts: {solution.objective_value}.")

Min. flux through GLCpts: 0.4794285714285715.


And consider custom constraints:

In [17]:
solution = sd.fba(model,constraints=['-EX_o2_e <= 5', 'ATPM = 20'])

print(f"Maximum growth at limited oxygen uptake and high ATP maintenance: {solution.objective_value}.")

Maximum growth at limited oxygen uptake and high ATP maintenance: 0.26305573292588313.


In [10]:
solution = sd.fva(model)
print(solution)

         minimum  maximum
PFK          0.0   176.61
PFL          0.0    40.00
PGI        -50.0    10.00
PGK        -20.0    -0.00
PGL          0.0    60.00
...          ...      ...
NADH16       0.0   120.00
NADTRHD      0.0   378.22
NH4t         0.0    10.00
O2t          0.0    60.00
PDH          0.0    40.00

[95 rows x 2 columns]


## Flux ranges (FVA)

## Yield optimization